

Extraemos los datos



In [1]:
pip install pandas

In [2]:
pip install kaggle

In [3]:
from google.colab import files
files.upload()  # subir el archivo kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"andrsbentezconcha","key":"1803aefc5414583d80ac97197ddb8271"}'}

In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"  # Ruta donde colocaste el archivo kaggle.json

In [5]:
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi

# Crear una instancia de la API de Kaggle y autenticar
api = KaggleApi()

# Descargar el conjunto de datos Wine Reviews de Kaggle
dataset_name = "zynicide/wine-reviews"  # Nombre correcto del dataset
download_path = "/content/wine-reviews.zip"  # Ruta de descarga del archivo ZIP

# Descargar el conjunto de datos
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# Mover los archivos a la carpeta '/content/wine-reviews'
destination_path = "/content/wine-reviews"
if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Mover los archivos descargados a la carpeta de destino
shutil.move(download_path, destination_path)

print(f"Archivos movidos a: {destination_path}")


Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
Archivos movidos a: /content/wine-reviews


In [6]:
# Ruta al archivo CSV que se descomprimió
csv_file_path = '/content/wine-reviews/winemag-data-130k-v2.csv'

se cargan los dos df

In [7]:
import pandas as pd

In [8]:
df1 = pd.read_csv("/content/wine-reviews/wine-reviews.zip/winemag-data_first150k.csv")
df2 = pd.read_csv("/content/wine-reviews/wine-reviews.zip/winemag-data-130k-v2.csv")

In [109]:
# Eliminar las filas con valores nulos en 'country' y 'province'
df1 = df1.dropna(subset=['country', 'province'])

In [110]:
# Eliminar las filas con valores nulos en 'country' y 'province'
df2 = df2.dropna(subset=['country', 'province'])

In [11]:
# Eliminar las filas con valores nulos en variety
df2 = df2.dropna(subset=['variety'])

In [12]:
df2['price'].fillna(df2['price'].median(), inplace=True)

<ipython-input-12-129ccf0a5d08>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['price'].fillna(df2['price'].median(), inplace=True)


In [13]:
# Crear una nueva columna para la relación calidad-precio
df2['quality_price_ratio'] = df2['points'] / df2['price']

In [14]:
# Eliminar las filas con datos faltantes en la columna 'region_1'
df2= df2.dropna(subset=['region_1'])

**Breve descripción de las variables**

**country**: El país de origen del vino.

**description**: Una descripción del vino, donde se detallan las características como sabor, aroma y notas de cata.

**designation**: Nombre o designación específica del vino, a menudo relacionado con la bodega o la subregión.

**points**: La puntuación otorgada al vino por los catadores, generalmente en una escala de 1 a 100.

**price**: El precio del vino, en la moneda local del país de origen.

**province**: La provincia o estado dentro del país donde se produce el vino.

**region_1**: Una región más específica dentro de la provincia, indicando la zona exacta de producción.

**region_2**: Una subregión aún más específica dentro de region_1, aunque a menudo está vacía.

**variety**: El tipo de uva (variedad) utilizada para el vino.

**winery**: El nombre de la bodega o productor del vino.

In [111]:
df2.shape

(108724, 15)

# 1. Dataframe definido para los modelos de recomendación

ya que tuve problemas con la RAM , vamos a particionar el df2 mediante la seleccion de los vinos de los 3 paises que mejor puntaje promedio tienen. Y además otra particion en precios 20% más bajos. con eso llego a un aprox de 5000 filas.

vinos en paises con mejor puntaje y menor precio

In [149]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack

denifimos el daframe particionado en vinos baratos y con los paises de mejor puntaje (con este df se van a generar los dos modelos, el de contenido y el colaborativo).

In [118]:
# Calcular el puntaje promedio por país
avg_points_by_country = df2.groupby('country')['points'].mean().sort_values(ascending=False)

In [120]:
avg_points_by_country

,points
country,
Canada,89.369650
France,88.850979
Australia,88.582725
US,88.573876
Italy,88.561677
Spain,87.286274
Argentina,86.721110


In [121]:
# Seleccionar los tres países con mayor puntaje promedio
top_countries = avg_points_by_country.head(3).index

In [122]:
top_countries

Index(['Canada', 'France', 'Australia'], dtype='object', name='country')

In [125]:
# Filtrar el DataFrame por los tres países con puntajes más altos
df_filtered = df2[df2['country'].isin(top_countries)]

In [128]:
# Filtrar los precios bajos (por ejemplo, seleccionamos el 20% inferior de los precios)
price_threshold = df_filtered['price'].quantile(0.2)
df_filtered = df_filtered[df_filtered['price'] <= price_threshold]

In [129]:
# Ver las primeras filas del DataFrame filtrado
df_filtered.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,quality_price_ratio
42,42,France,"This is a festive wine, with soft, ripe fruit ...",Nouveau,86,9.0,Beaujolais,Beaujolais,NaN,Roger Voss,@vossroger,Henry Fessy 2012 Nouveau (Beaujolais),Gamay,Henry Fessy,9.555556
49,49,France,"Soft and fruity, this is a generous, ripe wine...",Eté Indien,86,14.0,Beaujolais,Brouilly,NaN,Roger Voss,@vossroger,Vignerons de Bel Air 2011 Eté Indien (Brouilly),Gamay,Vignerons de Bel Air,6.142857
53,53,France,"Fruity and lightly herbaceous, this has fine t...",La Fleur d'Amélie,85,15.0,Bordeaux,Bordeaux Blanc,NaN,Roger Voss,@vossroger,Château de Sours 2011 La Fleur d'Amélie (Bord...,Bordeaux-style White Blend,Château de Sours,5.666667
66,66,France,"This soft, rounded wine is ripe with generous ...",NaN,86,15.0,Burgundy,Mâcon-Milly Lamartine,NaN,Roger Voss,@vossroger,Vignerons des Terres Secrètes 2015 Mâcon-Mill...,Chardonnay,Vignerons des Terres Secrètes,5.733333
82,82,France,"This fruity, sweet wine is immediately attract...",La Réserve,86,11.0,France Other,Vin de France,NaN,Roger Voss,@vossroger,Lionel Osmin & Cie 2016 La Réserve Petit Manse...,Petit Manseng,Lionel Osmin & Cie,7.818182


In [130]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5137 entries, 42 to 129894
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             5137 non-null   int64  
 1   country                5137 non-null   object 
 2   description            5137 non-null   object 
 3   designation            2961 non-null   object 
 4   points                 5137 non-null   int64  
 5   price                  5137 non-null   float64
 6   province               5137 non-null   object 
 7   region_1               5137 non-null   object 
 8   region_2               0 non-null      object 
 9   taster_name            4928 non-null   object 
 10  taster_twitter_handle  4925 non-null   object 
 11  title                  5137 non-null   object 
 12  variety                5137 non-null   object 
 13  winery                 5137 non-null   object 
 14  quality_price_ratio    5137 non-null   float64
dtypes: flo

In [131]:
df_filtered.shape

(5137, 15)

1.1 cargamos librerías

In [266]:
from surprise import Dataset, Reader
import numpy as np
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise import SVD
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

# 2.- Sistema de Recomendación de Vinos basado en contenido (Recomendación de vinos similares)

utilizo descpción y la vectorizo para utilizarla como criterio o contenido para recomendar vinos.

se va a recomendar con las variables: description', 'variety', 'region_1', 'point'

combinando las columnas description, variety y region_1 en una nueva columna text_combined para luego utilizarla en el vectorizador TF-IDF. Con esto se provecha varias características textuales para construir un vector representativo para cada vino.

# 2.1.- Preprocesamiento.

In [215]:
df_filtered['text_combined'] = df_filtered['description'].fillna('') + ' ' + df_filtered['variety'].fillna('') + ' ' + df_filtered['region_1'].fillna('') + ' ' + df_filtered['points'].astype(str)

transformar el texto combinado en una matriz de características numéricas

El parámetro max_features=5000 asegura que solo se mantendrán las 5000 características más relevantes (palabras o términos).

In [216]:
# Crear un vectorizador TF-IDF para las características combinadas
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
# Ajustar y transformar el texto
tfidf_matrix = vectorizer.fit_transform(df_filtered['text_combined'])

asegurando que solo las 5000 palabras más relevantes sean consideradas.

In [252]:
# Ver el tamaño de la matriz resultante
tfidf_matrix.shape

(5137, 5000)

Calcular la similitud de coseno para descripcion y utilizarla en el modelo de recomendación

In [218]:
# Calcular la similitud del coseno entre los vinos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Ver la similitud entre los primeros 5 vinos
cosine_sim[:5, :5]


array([[1.        , 0.17965174, 0.10086378, 0.10253914, 0.07986085],
       [0.17965174, 1.        , 0.0797967 , 0.22220233, 0.12279659],
       [0.10086378, 0.0797967 , 1.        , 0.09884771, 0.04431808],
       [0.10253914, 0.22220233, 0.09884771, 1.        , 0.05957071],
       [0.07986085, 0.12279659, 0.04431808, 0.05957071, 1.        ]])

Codificación de Variables Categóricas

In [269]:
# Codificar las categorías (variedad y región) si es necesario
label_encoder = LabelEncoder()
df_filtered['variety_encoded'] = label_encoder.fit_transform(df_filtered['variety'])
df_filtered['region_encoded'] = label_encoder.fit_transform(df_filtered['region_1'])

# 2.2.- red neuronal

Preparación para el Modelo

X (características) e y (puntuaciones).

objetivo: con la finalidad de predecir las puntuaciones de los vinos

In [270]:
# Convertir las características en una matriz (concatenar el TF-IDF con las variables codificadas)
X = np.hstack([tfidf_matrix.toarray(), df_filtered[['variety_encoded', 'region_encoded']].values])

In [271]:
y = df_filtered['points'].values

In [272]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2.3- rede neuronal, utilizando TensorFlow/Keras.

Construcción del modelo

In [273]:
# Crear el modelo de red neuronal
model = Sequential()

In [274]:
# Capa de entrada: el tamaño de la entrada es el número de características de X
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Capa de entrada con activación ReLU

# Dropout para evitar el sobreajuste
model.add(Dropout(0.2))

# Capa oculta
model.add(Dense(64, activation='relu'))  # Capa oculta

# Capa de salida: una salida para predecir la puntuación (en este caso usamos una salida continua)
model.add(Dense(1, activation='linear'))  # Salida continua (valoración del vino)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [275]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [276]:
# Resumen del modelo
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                     │ (None, 128)                 │         640,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 648,705 (2.47 MB)

 Trainable params: 648,705 (2.47 MB)

 Non-trainable params: 0 (0.00 B)

# 2.4- Entrenar el modelo

In [277]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 3841.3311 - mae: 54.8865 - val_loss: 1470.0653 - val_mae: 28.7788
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1371.2494 - mae: 28.1331 - val_loss: 619.7040 - val_mae: 19.0811
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 433.8063 - mae: 16.3924 - val_loss: 123.3630 - val_mae: 8.6889
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 127.7752 - mae: 8.8861 - val_loss: 71.3491 - val_mae: 6.4223
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 86.1882 - mae: 7.3428 - val_loss: 54.5569 - val_mae: 5.6312
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 74.0520 - mae: 6.8807 - val_loss: 45.0678 - val_mae: 5.1268
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 62.9848 - mae: 6.3731 - val_loss: 46.2929 - val_mae: 5.3126
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 57.5007 - mae: 5.9424 - val_loss: 37.8993 - val_mae: 4.8461
Epoch 9/10
129/129 ━━━━━━

 tiene 128 neuronas

 el número de parámetros (640,384) indica que cada entrada del modelo se conecta a cada una de estas 128 neuronas.

 La capa densa (con activación ReLU, normalmente) toma las características de entrada del vino (que fueron vectorizadas mediante TF-IDF u otro proceso de transformación de texto) y las procesa, buscando patrones complejos en la información. Esto ayudará a capturar representaciones abstractas de las características del vino, como las palabras clave en la descripción, las variedades y la región.

# 2.5.- evluar el modelo Error de test

In [384]:
# Evaluar el modelo
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error (MAE) en el conjunto de prueba: {mae}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 20.0662 - mae: 3.6119
Mean Absolute Error (MAE) en el conjunto de prueba: 3.5866174697875977


las predicciones de puntuaciones del vino están a 3.58 puntos de las puntuaciones reales.

El Mean Absolute Error (MAE) en el conjunto de prueba es de 3.58, lo que significa que, en promedio, la predicción de la puntuación del vino se desvía en 3.58 puntos de la puntuación real.

MAE: Un valor de 3.58 significa que el modelo tiene una desviación promedio de aproximadamente 3.58 puntos en la puntuación de los vinos.

# 2.6 - Validacion cruzada

Configurar la validación cruzada con KFold. (5 pliegues)

In [286]:
# Configurar validación cruzada (5 pliegues)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []

In [287]:
# Ejecutar validación cruzada
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Crear y entrenar el modelo
    model = create_model(X_train.shape[1])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predecir y calcular el MAE para este pliegue
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    mae_scores.append(mae)
    print(f'MAE para el pliegue: {mae}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MAE para el pliegue: 3.9723947048187256


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MAE para el pliegue: 3.9251227378845215


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MAE para el pliegue: 4.531281471252441


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MAE para el pliegue: 4.034364223480225


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MAE para el pliegue: 4.578387260437012


In [385]:
# Resultados finales
print(f"MAE promedio en los {kf.get_n_splits()} pliegues: {np.mean(mae_scores)}")
print(f"Desviación estándar del MAE: {np.std(mae_scores)}")

MAE promedio en los 5 pliegues: 4.208310079574585
Desviación estándar del MAE: 0.2854385871004461


* 4.21, lo que sugiere que el modelo es razonablemente consistente
* La desviación estándar de 0.29 indica que las predicciones no varían significativamente entre los diferentes pliegues de validación.

MAE promedio: 4.21 — Esto significa que, en promedio, el modelo tiene una desviación de 4.21 puntos en las predicciones de puntuación para los vinos. Es un buen valor, especialmente si tienes en cuenta que las puntuaciones suelen variar entre 0 y 100.

Desviación estándar: 0.29 — Esto muestra que el modelo tiene un rendimiento bastante consistente a través de los diferentes pliegues, ya que la desviación estándar es baja.

# 2.7- predecir la puntuación de un vino en particular

In [289]:
# Predecir la puntuación para un vino específico (suponiendo que el vino tiene un índice 12496)
wine_index = 0
wine_features = X[wine_index].reshape(1, -1)  # Convertir las características en una matriz 2D
predicted_score = model.predict(wine_features)
print(f'Predicción de puntuación para el vino con índice {wine_index}: {predicted_score[0][0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicción de puntuación para el vino con índice 0: 83.51055908203125


Este valor parece estar dentro del rango esperado (siendo las puntuaciones comunes entre 0 y 100), lo que sugiere que el modelo está funcionando bien.

# 2.8- recomendar los vinos más similares

In [291]:
# Obtener la predicción de puntuaciones para todos los vinos
predicted_scores = model.predict(X)

# Ordenar los vinos por las puntuaciones predichas
recommended_indices = np.argsort(predicted_scores.flatten())[-6:-1]  # Top 5 recomendaciones

# Mostrar las recomendaciones
recommended_wines = df_filtered.iloc[recommended_indices]
print(recommended_wines[['title', 'variety', 'region_1', 'price', 'points']])


161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
                                                    title  \
112164  Château du Rouët 2015 Coeur Estérelle Rosé (Cô...   
110680       Vieux Château des Combes 2014  Saint-Émilion   
13746   Clos Troteligotte 2012 C.Sb.V. White (Vin de P...   
65190   Calmel & Joseph 2014 Les Terroirs Red (Saint-C...   
37195   Lieubeau 2015 La Fruitière Vignes Blanches Whi...   

                         variety            region_1  price  points  
112164                      Rosé   Côtes de Provence   15.0      88  
110680  Bordeaux-style Red Blend       Saint-Émilion   15.0      90  
13746                White Blend  Vin de Pays du Lot   16.0      88  
65190      Rhône-style Red Blend       Saint-Chinian   15.0      88  
37195                White Blend        Val de Loire   15.0      88  


# 2.9 - CONCLUSIONES  - recomendación basada en contenido

* la red neuronal tuvo la finalidad de predecir las puntuaciones de los vinos.
* MAE (Error Absoluto Medio): El modelo mostró un MAE de 3.81 en el conjunto de prueba, lo que indica que, en promedio, las predicciones de puntuaciones del vino están a 3.81 puntos de las puntuaciones reales.
* la validación cruzada KFold (5 pliegues) obtuvo un 4.21, lo que sugiere que el modelo es razonablemente consistente. La desviación estándar de 0.29 indica que las predicciones no varían significativamente entre los diferentes pliegues de validación.
* Se realizó una predicción individual de puntuación para un vino específico, mostrando que el modelo puede hacer predicciones razonables para vinos no vistos durante el entrenamiento.
* La combinación de la similitud de coseno y la red neuronal permite generar recomendaciones de vinos que son similares en términos de características textuales y puntuación, lo que es útil para usuarios que buscan vinos con perfiles similares.


# 3.- SISTEMA DE RECOMENDACIÓN COLABORATIVO O BASADO EN USUARIOS

Sistema de Recomendación de Vinos colaborativo (Recomendación de catadores similares)

OBJETIVO: un modelo que recomiende vinos (title) a otros catadores basados en las calificaciones dadas por otros con gustos similares.

In [35]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505171 sha256=09615d4c07867f785fa97b5afb35a9b5d43e7f0c2250adf64b893b32e6ef6b99
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


creamos una tabla dinámica en la que cada fila es un taster_name y cada columna es un title de vino. Los valores en la tabla son 1 si el catador ha probado el vino, y 0 si no lo ha probado.

Uso de Surprise para el sistema colaborativo

In [292]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
from surprise import SVD
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score

# 3.1- Preprocesamiento

In [347]:
# Eliminar filas donde 'taster_name' es nulo
df_filtered_cleaned = df_filtered.dropna(subset=['taster_name'])
# Eliminar filas con 'taster_name' nulo en el DataFrame original
df_filtered.dropna(subset=['taster_name'], inplace=True)

# 3.1 - Definir parámetros.

In [335]:
# Crear el dataset adecuado para Surprise
reader = Reader(rating_scale=(df_filtered_cleaned['points'].min(), df_filtered_cleaned['points'].max()))
data = Dataset.load_from_df(df_filtered_cleaned[['taster_name', 'title', 'points']], reader)

dividir en entrenamientio y prueba.

In [336]:
# Dividir en conjunto de entrenamiento y conjunto de prueba
trainset, testset = train_test_split(data, test_size=0.2)

# 3.2- Configurar y entrenar el modelo KNN

creamos y entrenamos el modelo de KNN basado en usuarios para calcular las similitudes entre los catadores y recomendar vinos basados en estas similitudes.

la distancia coseno como métrica de similitud entre usuarios, y habilitas el filtrado basado en usuarios.

In [337]:
# Configurar el modelo KNN
sim_options = {
    'name': 'cosine',  # Usamos distancia coseno
    'user_based': True  # Filtrado basado en usuarios
}

In [338]:
# Crear el modelo KNN
model_knn = KNNBasic(sim_options=sim_options)

In [339]:
# Entrenar el modelo
model_knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


 Hacer predicciones

In [340]:
# Hacer predicciones sobre el conjunto de prueba
predictions_knn = model_knn.test(testset)

Evaluar el modelo con RMSE

In [341]:
# Evaluar el modelo con RMSE
rmse_knn = accuracy.rmse(predictions_knn)
print(f"RMSE de KNNBasic: {rmse_knn}")

RMSE: 1.9384
RMSE de KNNBasic: 1.9384223444842636


es relativamente bajo y sugiere que el modelo está funcionando bien.

un RMSE bajo significa que las predicciones de tu modelo están más cerca de las calificaciones reales.

Valor alto de RMSE: Indica que las predicciones están más alejadas de los valores reales y que el modelo tiene un mayor error.

Una vez que el modelo está entrenado, creamos una función que recomendará vinos a un catador específico, basándonos en los catadores más similares:

Función para recomendar vinos a un catador específico

In [365]:
# Función para recomendar vinos a un catador específico
def recomendar_vinos(catador, model_knn, trainset, top_n=5):
    # Obtener el índice interno del catador en el conjunto de entrenamiento
    inner_id = trainset.to_inner_uid(catador)

    # Obtener los vecinos más similares al catador
    neighbors = model_knn.get_neighbors(inner_id, k=top_n)

    # Almacenar las predicciones de vinos que los vecinos han probado
    recommended_vinos = []

    # Iterar sobre los vecinos para obtener los vinos que han calificado
    for neighbor in neighbors:
        # Obtener las predicciones de todos los vinos que el vecino ha calificado
        for (uid, iid, true_r) in trainset.all_ratings():
            if uid == neighbor:  # Si el vecino ha calificado este vino
                predicted_rating = model_knn.predict(uid, iid)  # Hacemos la predicción
                recommended_vinos.append((iid, predicted_rating.est))

    # Ordenar los vinos recomendados por la predicción de la puntuación (de mayor a menor)
    recommended_vinos = sorted(recommended_vinos, key=lambda x: x[1], reverse=True)

    # Mostrar los vinos recomendados
    print(f"\nVinos recomendados para el catador {catador}:")
    for i, (vino_id, puntuacion) in enumerate(recommended_vinos[:top_n]):
        # Convertir el ID del vino de vuelta a su título
        vino_title = trainset.to_raw_iid(vino_id)
        print(f"{i+1}. {vino_title}: Predicción de puntuación: {puntuacion:.2f}")


In [368]:
# Reemplaza 'Lauren Buzzeo' con el nombre de un catador que exista en tu conjunto de datos
recomendar_vinos('Lauren Buzzeo', model_knn, trainset, top_n=5)


Vinos recomendados para el catador Lauren Buzzeo:
1. Banrock Station 2015 Chardonnay (South Eastern Australia): Predicción de puntuación: 86.32
2. Shoofly 2008 The Freckle White (Adelaide): Predicción de puntuación: 86.32
3. Altoona Hills 2008 Rosé Kosher Shiraz (South Eastern Australia): Predicción de puntuación: 86.32
4. Robert Oatley 2010 Rosé of Sangiovese Rosé (Mudgee): Predicción de puntuación: 86.32
5. Alice White 2006 Cabernet Sauvignon (South Eastern Australia): Predicción de puntuación: 86.32


# 3.3.- evaluar con Singular Value Decompositionen SVD,

ahora la idea es reducir las dimensiones de la matriz de interacción (entre catadores y vinos) para encontrar patrones latentes que expliquen las calificaciones

Singular Value Decomposition

Definir parámetros.

In [354]:
# Leer datos desde el dataframe que contiene las calificaciones
reader = Reader(rating_scale=(df_filtered_cleaned['points'].min(), df_filtered_cleaned['points'].max()))
data = Dataset.load_from_df(df_filtered_cleaned[['taster_name', 'title', 'points']], reader)

Dividir en entrenamiento y prueba

In [355]:
# Dividir en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

Crear el modelo SVD

In [371]:
model_svd = SVD()

Entrenar el modelo

In [372]:
model_svd.fit(trainset)

Hacer predicciones sobre el conjunto de prueba

In [377]:
predictions_svd = model_svd.test(testset)

Evaluar el modelo con RMSE

In [379]:
rmse_svd = accuracy.rmse(predictions_svd)
print(f"RMSE de SVD: {rmse_svd}")

RMSE: 1.9898
RMSE de SVD: 1.9897813192051406


RMSE mide la diferencia promedio entre las calificaciones predichas por el modelo y las calificaciones reales de los usuarios en el conjunto de prueba.

Un RMSE más bajo significa que las predicciones del modelo son más precisas, ya que las diferencias entre las calificaciones predichas y las reales son menores

En este caso, el RMSE de 1.98 podría ser adecuado, dependiendo de las calificaciones en tu conjunto de datos y cómo se distribuyen.

# 3.4- con LightFM

In [329]:
pip install lightfm

matriz de interaccion

In [357]:
# df_filtered contiene 'taster_name', 'title' y 'points'
# Creamos un mapeo de los catadores y vinos a índices enteros
user_mapping = {user: idx for idx, user in enumerate(df_filtered['taster_name'].unique())}
item_mapping = {item: idx for idx, item in enumerate(df_filtered['title'].unique())}

# Convertimos las puntuaciones en una matriz dispersa
rows = df_filtered_cleaned['taster_name'].map(user_mapping)
cols = df_filtered_cleaned['title'].map(item_mapping)
values = df_filtered_cleaned['points']

interaction_matrix = coo_matrix((values, (rows, cols)), shape=(len(user_mapping), len(item_mapping)))


In [358]:
# Creamos un modelo híbrido (usando tanto contenido como colaborativo)
model_lightfm = LightFM(loss='warp')  # 'warp' es un tipo de pérdida eficiente para recomendaciones

Entrenamos el modelo

In [359]:
# Entrenamos el modelo
model_lightfm.fit(interaction_matrix, epochs=30, num_threads=4)

evaluacion con AUC (Área bajo la curva ROC)

In [382]:
# Evaluación utilizando AUC
train_auc = auc_score(model_lightfm, interaction_matrix).mean()
print(f"AUC de entrenamiento: {train_auc}")

AUC de entrenamiento: 0.9920408725738525


KNN es más simple y trabaja directamente con la similitud entre usuarios, mientras que LightFM combina características de los ítems y las interacciones, lo que puede resultar en un rendimiento más robusto si tienes características adicionales de los vinos o catadores.

Un valor de AUC cercano a 1 es ideal. algo cerca de 0.5, eso indica que el modelo no es mejor que una predicción aleatoria

Si el AUC de LightFM es significativamente más alto que el RMSE de KNN, es probable que LightFM esté funcionando mejor en este caso.

 Función para recomendar vinos con el modelo LightFM

In [383]:
import numpy as np

def recomendar_vinos_lightfm(catador, model, interaction_matrix, user_mapping, item_mapping, top_n=5):
    # Obtener el índice del catador en el mapeo
    user_idx = user_mapping.get(catador)

    if user_idx is None:
        print(f"Catador {catador} no encontrado.")
        return

    # Obtener las predicciones para todos los vinos para el catador
    # El modelo LightFM devuelve las puntuaciones predichas para cada vino en el sistema
    scores = model.predict(user_idx, np.arange(interaction_matrix.shape[1]))

    # Filtrar los vinos que el catador ya ha calificado (no los recomendamos)
    rated_items = interaction_matrix.getrow(user_idx).nonzero()[1]  # Índices de vinos ya calificados
    recommended_items = [(item, score) for item, score in zip(np.arange(interaction_matrix.shape[1]), scores) if item not in rated_items]

    # Ordenar las recomendaciones por la puntuación (de mayor a menor)
    recommended_items = sorted(recommended_items, key=lambda x: x[1], reverse=True)

    # Mostrar los vinos recomendados
    print(f"\nVinos recomendados para el catador {catador}:")
    for i, (item_idx, score) in enumerate(recommended_items[:top_n]):
        # Convertir el índice del vino en su nombre
        vino_title = list(item_mapping.keys())[list(item_mapping.values()).index(item_idx)]
        print(f"{i+1}. {vino_title}: Predicción de puntuación: {score:.2f}")

# Usar la función para recomendar vinos a un catador
recomendar_vinos_lightfm('Lauren Buzzeo', model_lightfm, interaction_matrix, user_mapping, item_mapping, top_n=5)



Vinos recomendados para el catador Lauren Buzzeo:
1. Michel Laroche 2001 Merlot (Vin de Pays d'Oc): Predicción de puntuación: -2.03
2. Quails' Gate 2009 Chardonnay (Okanagan Valley): Predicción de puntuación: -2.24
3. Cave Spring 2011 Riesling (Niagara Peninsula): Predicción de puntuación: -2.57
4. North 42 Degrees 2016 Sweet Riesling (Lake Erie North Shore): Predicción de puntuación: -2.63
5. Abarbanel 2001 Estate Bottled Chardonnay (Vin de Pays de L'Aude): Predicción de puntuación: -2.69


# 4.- CONCLUSIONES - recomendación colaborativa

* El objetivo general de este sistema de recomendación es recomendar vinos a los catadores basándose en las calificaciones previas dadas por otros catadores
* KNN basado en usuarios (Filtrado colaborativo) RMSE (Root Mean Square Error): El RMSE obtenido fue relativamente bajo, lo que sugiere que el modelo ha sido capaz de hacer predicciones razonablemente buenas
* SVD (Descomposición en valores singulares)El RMSE para el modelo SVD fue de 1.98, lo que puede considerarse adecuado. Un RMSE más bajo indica que el modelo ha logrado aprender patrones subyacentes que permiten hacer mejores predicciones sobre las calificaciones que los catadores darían a los vinos que no han probado.
* LightFM (Modelo híbrido) Un valor cercano a 1 en AUC es ideal, lo que indica que el modelo tiene una buena capacidad para diferenciar entre interacciones positivas (vinos que un catador probablemente disfrutaría) y negativas (vinos que no le gustarían).